In [1]:
!pip install -q sagemaker sagemaker[local]

# Install `transformers` from master
!pip install -q git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers|sagemaker'

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker           1.60.2    
tokenizers          0.7.0     
transformers        2.11.0    
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-transformer'

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


In [3]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!mkdir EsperBERTo

!wget -O EsperBERTo/oscar.eo.txt -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt 


--2020-06-10 20:01:15--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)...52.85.193.210, 52.85.193.66, 52.85.193.97, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|52.85.193.210|:443... connected.
HTTP request sent, awaiting response...200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘EsperBERTo/oscar.eo.txt’

EsperBERTo/oscar.eo 100%[===================>] 298.25M  60.5MB/s    in 5.1s    

2020-06-10 20:01:22 (58.8 MB/s) - ‘EsperBERTo/oscar.eo.txt’ saved [312733741/312733741]



In [4]:
inputs = sagemaker_session.upload_data(path='EsperBERTo', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-transformer


In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    #train_instance_type='local',
                    source_dir='code',
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

In [17]:
%%time
estimator.fit({'training': inputs})

ple of Tensors, module_name:roberta.encoder.layer.1.attention.self NoneType
[2020-06-10 12:52:43.504 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.1.attention.self NoneType
[2020-06-10 12:52:43.504 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.1.attention NoneType
[2020-06-10 12:52:43.505 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.1 NoneType
[2020-06-10 12:52:43.505 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.1 NoneType
[2020-06-10 12:52:43.505 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.1 NoneType
[2020-06-10 12:52:43.506 algo-1:52 WARNING hook.py:870] var is not Tensor or list or tuple of Tensors, module_name:roberta.encoder.layer.2.attention.self NoneT

In [19]:
training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
# s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-10-11-15-27-506/output/model.tar.gz

s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-10-11-15-27-506/output/model.tar.gz


In [20]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [29]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']

model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='infer.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

In [30]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

----------------!pytorch-inference-2020-06-10-14-03-25-103


In [57]:
#FIXME: This works, but the predictor using endpoint does not work
input = {
    'text': "La suno <mask>."
}
response = predictor.predict(input)
print(response)

TypeError: __init__() missing 1 required positional argument: 'sagemaker_session'

In [56]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

input = {
    'text': "La suno <mask>."
}

content_type = "application/json "                                     
accept = "application/json"                                       
payload = json.dumps(input)

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response['Body'].read())  

b'[{"sequence": "<s> La suno estis.</s>", "score": 0.010310359299182892, "token": 394}, {"sequence": "<s> La suno estas.</s>", "score": 0.00764045724645257, "token": 316}, {"sequence": "<s> La suno diris.</s>", "score": 0.00608186237514019, "token": 963}, {"sequence": "<s> La suno iris.</s>", "score": 0.0037596658803522587, "token": 2480}, {"sequence": "<s> La suno,.</s>", "score": 0.003713993588462472, "token": 16}]'


In [28]:
sagemaker_session.delete_endpoint(predictor.endpoint)